# MetOx prediction using MusiteDeep

In this notebook, we will use the framework MusiteDeep to try to solve the methionine-oxidation problem. We will pre-train the MusiteDeep model on general phosphorylation data, and then we will fine-tune it using our methionine-oxidation dataset.

## Test scripts

Bash scripts to perform simple executions of the python scripts from MusiteDeep. We also try to control the GPU memory allocation, by including some statements in the main section of every executed script, as the GPU memory management seems to be specific for each keras session.

In [1]:
!python -V

Python 2.7.15 :: Anaconda, Inc.


In [2]:
%%time
# Predict
!./ssh_test_predict_custom.sh

Using TensorFlow backend.
2018-11-18 23:09:57.597485: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2018-11-18 23:09:57.669853: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:964] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2018-11-18 23:09:57.670341: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1432] Found device 0 with properties: 
name: GeForce GTX 1080 Ti major: 6 minor: 1 memoryClockRate(GHz): 1.6575
pciBusID: 0000:01:00.0
totalMemory: 10.91GiB freeMemory: 6.45GiB
2018-11-18 23:09:57.670369: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1511] Adding visible gpu devices: 0
2018-11-18 23:09:57.853213: I tensorflow/core/common_runtime/gpu/gpu_device.cc:982] Device interconnect StreamExecutor with strength 1 edge matrix:
2018-11-18 23:09:57.853254: I tensorflow/co

In [ ]:
%%time
# Fine-tuning
!./ssh_test_fine_tune.sh

# Custom general model

We now pre-train a custom general model on the general phosphorylation training dataset.

Firstly, we check the total number of resiudes contained in the dataset:

In [3]:
# Positive residues (in the original article, 36284 are reported)
!cat ../../testdata/training_proteins_nonredundant_STY.fasta | grep ">sp" -v | grep "#" -o | wc -l

36284


In [4]:
# Positive and negative residues (in the original article, 841448 are reported)
!cat ../../testdata/training_proteins_nonredundant_STY.fasta | grep ">sp" -v | grep "S|T|Y" -Eo | wc -l

840198


In [26]:
# Negative Tyr residues (in the original article, 128007 are reported)
a = !cat ../../testdata/training_proteins_nonredundant_STY.fasta | grep ">sp" -v | grep "Y" -o | wc -l
p = !cat ../../testdata/training_proteins_nonredundant_STY.fasta | grep ">sp" -v | grep "Y#" -o | wc -l
int(a[0]) - int(p[0])

126757

Regarding the dataset described in the original article, this dataset lacks of 1250 Tyr negative sites.

General model pre-trained on S, T residues:

In [ ]:
%%time
!python ../../MusiteDeep_Keras2.0/MusiteDeep/train_general.py -input ../../testdata/training_proteins_nonredundant_STY.fasta -output-prefix models/pre-train/custom_general_ST -residue-types S,T -nclass=5

General model pre-trained on S, T, Y residues:

In [ ]:
%%time
!python ../../MusiteDeep_Keras2.0/MusiteDeep/train_general.py -input ../../testdata/training_proteins_nonredundant_STY.fasta -output-prefix models/pre-train/custom_general_STY -residue-types S,T,Y -nclass=5

## Using all phoshorylation data

We now pre-train a custom general model using both training and testing general phoshorylation datasets.

First of all, we check that there are not proteins contained both in training and testing datasets, based on their IDs:

In [25]:
!cat ../../testdata/training_testing_proteins_nonredundant_STY.fasta | grep ">sp" | wc -l

7627


In [27]:
!cat ../../testdata/training_testing_proteins_nonredundant_STY.fasta | grep ">sp" | sort | uniq | wc -l

7627


Then, we check the total number of resiudes contained in the dataset:

In [1]:
# Positive residues (in the original article, 38405 are reported)
!cat ../../testdata/training_testing_proteins_nonredundant_STY.fasta | grep ">sp" -v | grep "#" -o | wc -l

38405


In [11]:
# Tyr positive residues (in the original article, 1930 are reported)
!cat ../../testdata/training_testing_proteins_nonredundant_STY.fasta | grep ">sp" -v | grep "Y#" -o | wc -l

1930


In [24]:
# Tyr negative residues (in the original article, 137181 are reported)
a = !cat ../../testdata/training_testing_proteins_nonredundant_STY.fasta | grep ">sp" -v | grep "Y" -o | wc -l
p = !cat ../../testdata/training_testing_proteins_nonredundant_STY.fasta | grep ">sp" -v | grep "Y#" -o | wc -l
int(a[0]) - int(p[0])

135931

In [2]:
# Positive and negative residues (in the original article, 913623 are reported)
!cat ../../testdata/training_testing_proteins_nonredundant_STY.fasta | grep ">sp" -v | grep -Eo "S|T|Y" | wc -l

912373


Regarding the dataset described in the original article, this dataset lacks of the 1250 Tyr negative sites of the training set, as stated above.

General model pre-trained on S, T residues:

In [ ]:
%%time
!python ../../MusiteDeep_Keras2.0/MusiteDeep/train_general.py -input ../../testdata/training_testing_proteins_nonredundant_STY.fasta -output-prefix all-phos-data/models/pre-train/custom_general_ST -residue-types S,T -nclass=5

General model pre-trained on S, T, Y residues:

In [ ]:
%%time
!python ../../MusiteDeep_Keras2.0/MusiteDeep/train_general.py -input ../../testdata/training_testing_proteins_nonredundant_STY.fasta -output-prefix all-phos-data/models/pre-train/custom_general_STY -residue-types S,T,Y -nclass=5

# Fine-tuned models

We first fine-tune the custom general models using default hyper-parameters configurations.

## Using repeated holdout strategy

Models are fine-tuned on MetOx data.

### Residues S, T

Using nclass=5, 

In [1]:
!cat train_bootstrap_custom_model_ST_5.sh

#!/bin/bash

for i in {1..10}
do
	# Train
	# Using repeated holdout strategy
	# Using the custom general model pre-trained on S, T residues
	# Using nclass=5 (default)
	# - initializing only the weights in last layer (for transfer learning)
	python ../../MusiteDeep_Keras2.0/MusiteDeep/train_kinase.py -input ../fasta_files/all_train_MetOx_$i.fasta -background-prefix models/pre-train/custom_general_ST -output-prefix models/fine-tune/rephold-ST-nclass5/metox_model_custom_general_ST_$i -residue-types M -transferlayer 1

	# Test
	python ../../MusiteDeep_Keras2.0/MusiteDeep/predict.py -input ../fasta_files/all_test_MetOx_$i.fasta -predict-type custom -model-prefix models/fine-tune/rephold-ST-nclass5/metox_model_custom_general_ST_$i -output results/rephold-ST-nclass5/metox_result_custom_general_ST_$i
done


In [ ]:
%%time
!./train_bootstrap_custom_model_ST_5.sh

Using nclass=1:

In [2]:
!cat train_bootstrap_custom_model_ST_1.sh

#!/bin/bash

for i in {1..10}
do
	# Train
	# Using repeated holdout strategy
	# Using the custom general model pre-trained on S, T residues
	# Using nclass=1
	# - initializing only the weights in last layer (for transfer learning)
	python ../../MusiteDeep_Keras2.0/MusiteDeep/train_kinase.py -input ../fasta_files/all_train_MetOx_$i.fasta -background-prefix models/pre-train/custom_general_ST -output-prefix models/fine-tune/rephold-ST-nclass1/metox_model_custom_general_ST_$i -residue-types M -nclass 1 -transferlayer 1

	# Test
	python ../../MusiteDeep_Keras2.0/MusiteDeep/predict.py -input ../fasta_files/all_test_MetOx_$i.fasta -predict-type custom -model-prefix models/fine-tune/rephold-ST-nclass1/metox_model_custom_general_ST_$i -output results/rephold-ST-nclass1/metox_result_custom_general_ST_$i
done


In [ ]:
%%time
!./train_bootstrap_custom_model_ST_1.sh

#### Fasta files with no initial M

Using nclass=1:

In [16]:
!cat musite-fasta-M/train_bootstrap_custom_model_ST_1.sh

#!/bin/bash

for i in {1..10}
do
	# Train
	# Using repeated holdout strategy
	# Using the custom general model pre-trained on S, T residues
	# Using nclass=1
	# - initializing only the weights in last layer (for transfer learning)
	python ../../MusiteDeep_Keras2.0/MusiteDeep/train_kinase.py -input fasta_files_M/all_train_MetOx_$i.fasta -background-prefix models/pre-train/custom_general_ST -output-prefix musite-fasta-M/models/fine-tune/rephold-ST-nclass1/metox_model_custom_general_ST_$i -residue-types M -nclass 1 -transferlayer 1

	# Test
	python ../../MusiteDeep_Keras2.0/MusiteDeep/predict.py -input fasta_files_M/all_test_MetOx_$i.fasta -predict-type custom -model-prefix musite-fasta-M/models/fine-tune/rephold-ST-nclass1/metox_model_custom_general_ST_$i -output musite-fasta-M/results/rephold-ST-nclass1/metox_result_custom_general_ST_$i
done


In [ ]:
%%time
!musite-fasta-M/train_bootstrap_custom_model_ST_1.sh

### Residues S, T, Y

Using nclass=5:

In [8]:
!cat train_bootstrap_custom_model_STY_5.sh

#!/bin/bash

for i in {1..10}
do
	# Train
	# Using repeated holdout strategy
	# Using the custom general model pre-trained on S, T, Y residues
	# Using nclass=5 (default)
	# - initializing only the weights in last layer (for transfer learning)
	python ../../MusiteDeep_Keras2.0/MusiteDeep/train_kinase.py -input ../fasta_files/all_train_MetOx_$i.fasta -background-prefix models/pre-train/custom_general_STY -output-prefix models/fine-tune/rephold-STY-nclass5/metox_model_custom_general_STY_$i -residue-types M -transferlayer 1

	# Test
	python ../../MusiteDeep_Keras2.0/MusiteDeep/predict.py -input ../fasta_files/all_test_MetOx_$i.fasta -predict-type custom -model-prefix models/fine-tune/rephold-STY-nclass5/metox_model_custom_general_STY_$i -output results/rephold-STY-nclass5/metox_result_custom_general_STY_$i
done


In [ ]:
%%time
!./train_bootstrap_custom_model_STY_5.sh

Using nclass=1:

In [2]:
!cat train_bootstrap_custom_model_STY_1.sh

#!/bin/bash

for i in {1..10}
do
	# Train
	# Using repeated holdout strategy
	# Using the custom general model pre-trained on S, T, Y residues
	# Using nclass=1
	# - initializing only the weights in last layer (for transfer learning)
	python ../../MusiteDeep_Keras2.0/MusiteDeep/train_kinase.py -input ../fasta_files/all_train_MetOx_$i.fasta -background-prefix models/pre-train/custom_general_STY -output-prefix models/fine-tune/rephold-STY-nclass1/metox_model_custom_general_STY_$i -residue-types M -nclass 1 -transferlayer 1

	# Test
	python ../../MusiteDeep_Keras2.0/MusiteDeep/predict.py -input ../fasta_files/all_test_MetOx_$i.fasta -predict-type custom -model-prefix models/fine-tune/rephold-STY-nclass1/metox_model_custom_general_STY_$i -output results/rephold-STY-nclass1/metox_result_custom_general_STY_$i
done


In [ ]:
%%time
!./train_bootstrap_custom_model_STY_1.sh

#### Learning-rate

Using lr=0.00075:

In [12]:
!cat lr/train_bootstrap_custom_model_STY_1_lr_00075.sh

#!/bin/bash

for i in {1..10}
do
	# Train
	# Using repeated holdout strategy
	# Using the custom general model pre-trained on S, T, Y residues
	# Using nclass=1 and lr=0.00075
	# - initializing only the weights in last layer (for transfer learning)
	python ../../MusiteDeep_Keras2.0/MusiteDeep/train_kinase.py -input ../fasta_files/all_train_MetOx_$i.fasta -background-prefix models/pre-train/custom_general_STY -output-prefix lr/models/fine-tune/rephold-STY-nclass1-lr00075/metox_model_custom_general_STY_$i -residue-types M -nclass 1 -transferlayer 1 -lr 0.00075

	# Test
	python ../../MusiteDeep_Keras2.0/MusiteDeep/predict.py -input ../fasta_files/all_test_MetOx_$i.fasta -predict-type custom -model-prefix lr/models/fine-tune/rephold-STY-nclass1-lr00075/metox_model_custom_general_STY_$i -output lr/results/rephold-STY-nclass1-lr00075/metox_result_custom_general_STY_$i
done


In [ ]:
%%time
!lr/train_bootstrap_custom_model_STY_1_lr_00075.sh

Using lr=0.0005:

In [3]:
!cat lr/train_bootstrap_custom_model_STY_1_lr_0005.sh

#!/bin/bash

for i in {1..10}
do
	# Train
	# Using repeated holdout strategy
	# Using the custom general model pre-trained on S, T, Y residues
	# Using nclass=1 and lr=0.0005
	# - initializing only the weights in last layer (for transfer learning)
	python ../../MusiteDeep_Keras2.0/MusiteDeep/train_kinase.py -input ../fasta_files/all_train_MetOx_$i.fasta -background-prefix models/pre-train/custom_general_STY -output-prefix lr/models/fine-tune/rephold-STY-nclass1-lr0005/metox_model_custom_general_STY_$i -residue-types M -nclass 1 -transferlayer 1 -lr 0.0005

	# Test
	python ../../MusiteDeep_Keras2.0/MusiteDeep/predict.py -input ../fasta_files/all_test_MetOx_$i.fasta -predict-type custom -model-prefix lr/models/fine-tune/rephold-STY-nclass1-lr0005/metox_model_custom_general_STY_$i -output lr/results/rephold-STY-nclass1-lr0005/metox_result_custom_general_STY_$i
done


In [ ]:
%%time
!lr/train_bootstrap_custom_model_STY_1_lr_0005.sh

Using lr=0.0003:

In [3]:
!cat lr/train_bootstrap_custom_model_STY_1_lr_0003.sh

#!/bin/bash

for i in {1..10}
do
	# Train
	# Using repeated holdout strategy
	# Using the custom general model pre-trained on S, T, Y residues
	# Using nclass=1 and lr=0.0003
	# - initializing only the weights in last layer (for transfer learning)
	python ../../MusiteDeep_Keras2.0/MusiteDeep/train_kinase.py -input ../fasta_files/all_train_MetOx_$i.fasta -background-prefix models/pre-train/custom_general_STY -output-prefix lr/models/fine-tune/rephold-STY-nclass1-lr0003/metox_model_custom_general_STY_$i -residue-types M -nclass 1 -transferlayer 1 -lr 0.0003

	# Test
	python ../../MusiteDeep_Keras2.0/MusiteDeep/predict.py -input ../fasta_files/all_test_MetOx_$i.fasta -predict-type custom -model-prefix lr/models/fine-tune/rephold-STY-nclass1-lr0003/metox_model_custom_general_STY_$i -output lr/results/rephold-STY-nclass1-lr0003/metox_result_custom_general_STY_$i
done


In [ ]:
%%time
!lr/train_bootstrap_custom_model_STY_1_lr_0003.sh

Using lr=0.0001:

In [6]:
!cat lr/train_bootstrap_custom_model_STY_1_lr_0001.sh

#!/bin/bash

for i in {1..10}
do
	# Train
	# Using repeated holdout strategy
	# Using the custom general model pre-trained on S, T, Y residues
	# Using nclass=1 and lr=0.0001
	# - initializing only the weights in last layer (for transfer learning)
	python ../../MusiteDeep_Keras2.0/MusiteDeep/train_kinase.py -input ../fasta_files/all_train_MetOx_$i.fasta -background-prefix models/pre-train/custom_general_STY -output-prefix lr/models/fine-tune/rephold-STY-nclass1-lr0001/metox_model_custom_general_STY_$i -residue-types M -nclass 1 -transferlayer 1 -lr 0.0001

	# Test
	python ../../MusiteDeep_Keras2.0/MusiteDeep/predict.py -input ../fasta_files/all_test_MetOx_$i.fasta -predict-type custom -model-prefix lr/models/fine-tune/rephold-STY-nclass1-lr0001/metox_model_custom_general_STY_$i -output lr/results/rephold-STY-nclass1-lr0001/metox_result_custom_general_STY_$i
done


In [ ]:
%%time
!lr/train_bootstrap_custom_model_STY_1_lr_0001.sh

Using lr=0.005:

In [8]:
!cat lr/train_bootstrap_custom_model_STY_1_lr_005.sh

#!/bin/bash

for i in {1..10}
do
	# Train
	# Using repeated holdout strategy
	# Using the custom general model pre-trained on S, T, Y residues
	# Using nclass=1 and lr=0.005
	# - initializing only the weights in last layer (for transfer learning)
	python ../../MusiteDeep_Keras2.0/MusiteDeep/train_kinase.py -input ../fasta_files/all_train_MetOx_$i.fasta -background-prefix models/pre-train/custom_general_STY -output-prefix lr/models/fine-tune/rephold-STY-nclass1-lr005/metox_model_custom_general_STY_$i -residue-types M -nclass 1 -transferlayer 1 -lr 0.005

	# Test
	python ../../MusiteDeep_Keras2.0/MusiteDeep/predict.py -input ../fasta_files/all_test_MetOx_$i.fasta -predict-type custom -model-prefix lr/models/fine-tune/rephold-STY-nclass1-lr005/metox_model_custom_general_STY_$i -output lr/results/rephold-STY-nclass1-lr005/metox_result_custom_general_STY_$i
done


In [ ]:
%%time
!lr/train_bootstrap_custom_model_STY_1_lr_005.sh

Using lr=0.01:

In [10]:
!cat lr/train_bootstrap_custom_model_STY_1_lr_01.sh

#!/bin/bash

for i in {1..10}
do
	# Train
	# Using repeated holdout strategy
	# Using the custom general model pre-trained on S, T, Y residues
	# Using nclass=1 and lr=0.01
	# - initializing only the weights in last layer (for transfer learning)
	python ../../MusiteDeep_Keras2.0/MusiteDeep/train_kinase.py -input ../fasta_files/all_train_MetOx_$i.fasta -background-prefix models/pre-train/custom_general_STY -output-prefix lr/models/fine-tune/rephold-STY-nclass1-lr01/metox_model_custom_general_STY_$i -residue-types M -nclass 1 -transferlayer 1 -lr 0.01

	# Test
	python ../../MusiteDeep_Keras2.0/MusiteDeep/predict.py -input ../fasta_files/all_test_MetOx_$i.fasta -predict-type custom -model-prefix lr/models/fine-tune/rephold-STY-nclass1-lr01/metox_model_custom_general_STY_$i -output lr/results/rephold-STY-nclass1-lr01/metox_result_custom_general_STY_$i
done


In [ ]:
%%time
!lr/train_bootstrap_custom_model_STY_1_lr_01.sh

#### Transfer-leayer

Using transfer-leayer=0:

In [16]:
!cat transfer-leayer/train_bootstrap_custom_model_STY_1_tl_0.sh

#!/bin/bash

for i in {1..10}
do
	# Train
	# Using repeated holdout strategy
	# Using the custom general model pre-trained on S, T, Y residues
	# Using nclass=1 and transfer-leayer=0
	# - initializing only the weights in last layer (for transfer learning)
	python ../../MusiteDeep_Keras2.0/MusiteDeep/train_kinase.py -input ../fasta_files/all_train_MetOx_$i.fasta -background-prefix models/pre-train/custom_general_STY -output-prefix transfer-leayer/models/rephold-STY-nclass1-tl0/metox_model_custom_general_STY_$i -residue-types M -nclass 1 -transferlayer 0

	# Test
	python ../../MusiteDeep_Keras2.0/MusiteDeep/predict.py -input ../fasta_files/all_test_MetOx_$i.fasta -predict-type custom -model-prefix transfer-leayer/models/rephold-STY-nclass1-tl0/metox_model_custom_general_STY_$i -output transfer-leayer/results/rephold-STY-nclass1-tl0/metox_result_custom_general_STY_$i
done


In [ ]:
%%time
!transfer-leayer/train_bootstrap_custom_model_STY_1_tl_0.sh

Using transfer-leayer=3:

In [18]:
!cat transfer-leayer/train_bootstrap_custom_model_STY_1_tl_3.sh

#!/bin/bash

for i in {1..10}
do
	# Train
	# Using repeated holdout strategy
	# Using the custom general model pre-trained on S, T, Y residues
	# Using nclass=1 and transfer-leayer=3
	# - initializing only the weights in last layer (for transfer learning)
	python ../../MusiteDeep_Keras2.0/MusiteDeep/train_kinase.py -input ../fasta_files/all_train_MetOx_$i.fasta -background-prefix models/pre-train/custom_general_STY -output-prefix transfer-leayer/models/rephold-STY-nclass1-tl3/metox_model_custom_general_STY_$i -residue-types M -nclass 1 -transferlayer 3

	# Test
	python ../../MusiteDeep_Keras2.0/MusiteDeep/predict.py -input ../fasta_files/all_test_MetOx_$i.fasta -predict-type custom -model-prefix transfer-leayer/models/rephold-STY-nclass1-tl3/metox_model_custom_general_STY_$i -output transfer-leayer/results/rephold-STY-nclass1-tl3/metox_result_custom_general_STY_$i
done


In [ ]:
%%time
!transfer-leayer/train_bootstrap_custom_model_STY_1_tl_3.sh

Using transfer-leayer=5:

In [25]:
!cat transfer-leayer/train_bootstrap_custom_model_STY_1_tl_5.sh

#!/bin/bash

for i in {1..10}
do
	# Train
	# Using repeated holdout strategy
	# Using the custom general model pre-trained on S, T, Y residues
	# Using nclass=1 and transfer-leayer=5
	# - initializing only the weights in last layer (for transfer learning)
	python ../../MusiteDeep_Keras2.0/MusiteDeep/train_kinase.py -input ../fasta_files/all_train_MetOx_$i.fasta -background-prefix models/pre-train/custom_general_STY -output-prefix transfer-leayer/models/rephold-STY-nclass1-tl5/metox_model_custom_general_STY_$i -residue-types M -nclass 1 -transferlayer 5

	# Test
	python ../../MusiteDeep_Keras2.0/MusiteDeep/predict.py -input ../fasta_files/all_test_MetOx_$i.fasta -predict-type custom -model-prefix transfer-leayer/models/rephold-STY-nclass1-tl5/metox_model_custom_general_STY_$i -output transfer-leayer/results/rephold-STY-nclass1-tl5/metox_result_custom_general_STY_$i
done


In [ ]:
%%time
!transfer-leayer/train_bootstrap_custom_model_STY_1_tl_5.sh

#### Transfer-leayer and learning-rate

Using transfer-leayer=0 and lr=0.00075:

In [27]:
!cat transfer-leayer/train_bootstrap_custom_model_STY_1_tl_0_lr_00075.sh

#!/bin/bash

for i in {1..10}
do
	# Train
	# Using repeated holdout strategy
	# Using the custom general model pre-trained on S, T, Y residues
	# Using nclass=1, transfer-leayer=0 and lr=0.00075
	# - initializing only the weights in last layer (for transfer learning)
	python ../../MusiteDeep_Keras2.0/MusiteDeep/train_kinase.py -input ../fasta_files/all_train_MetOx_$i.fasta -background-prefix models/pre-train/custom_general_STY -output-prefix transfer-leayer/models/rephold-STY-nclass1-tl0-lr00075/metox_model_custom_general_STY_$i -residue-types M -nclass 1 -transferlayer 0 -lr 0.00075

	# Test
	python ../../MusiteDeep_Keras2.0/MusiteDeep/predict.py -input ../fasta_files/all_test_MetOx_$i.fasta -predict-type custom -model-prefix transfer-leayer/models/rephold-STY-nclass1-tl0-lr00075/metox_model_custom_general_STY_$i -output transfer-leayer/results/rephold-STY-nclass1-tl0-lr00075/metox_result_custom_general_STY_$i
done


In [ ]:
%%time
!transfer-leayer/train_bootstrap_custom_model_STY_1_tl_0_lr_00075.sh

Using transfer-leayer=0 and lr=0.0005:

In [29]:
!cat transfer-leayer/train_bootstrap_custom_model_STY_1_tl_0_lr_0005.sh

#!/bin/bash

for i in {1..10}
do
	# Train
	# Using repeated holdout strategy
	# Using the custom general model pre-trained on S, T, Y residues
	# Using nclass=1, transfer-leayer=0 and lr=0.0005
	# - initializing only the weights in last layer (for transfer learning)
	python ../../MusiteDeep_Keras2.0/MusiteDeep/train_kinase.py -input ../fasta_files/all_train_MetOx_$i.fasta -background-prefix models/pre-train/custom_general_STY -output-prefix transfer-leayer/models/rephold-STY-nclass1-tl0-lr0005/metox_model_custom_general_STY_$i -residue-types M -nclass 1 -transferlayer 0 -lr 0.0005

	# Test
	python ../../MusiteDeep_Keras2.0/MusiteDeep/predict.py -input ../fasta_files/all_test_MetOx_$i.fasta -predict-type custom -model-prefix transfer-leayer/models/rephold-STY-nclass1-tl0-lr0005/metox_model_custom_general_STY_$i -output transfer-leayer/results/rephold-STY-nclass1-tl0-lr0005/metox_result_custom_general_STY_$i
done


In [ ]:
%%time
!transfer-leayer/train_bootstrap_custom_model_STY_1_tl_0_lr_0005.sh

Using transfer-leayer=0 and lr=0.0001:

In [1]:
!cat transfer-leayer/train_bootstrap_custom_model_STY_1_tl_0_lr_0001.sh

#!/bin/bash

for i in {1..10}
do
	# Train
	# Using repeated holdout strategy
	# Using the custom general model pre-trained on S, T, Y residues
	# Using nclass=1, transfer-leayer=0 and lr=0.0001
	# - initializing only the weights in last layer (for transfer learning)
	python ../../MusiteDeep_Keras2.0/MusiteDeep/train_kinase.py -input ../fasta_files/all_train_MetOx_$i.fasta -background-prefix models/pre-train/custom_general_STY -output-prefix transfer-leayer/models/rephold-STY-nclass1-tl0-lr0001/metox_model_custom_general_STY_$i -residue-types M -nclass 1 -transferlayer 0 -lr 0.0001

	# Test
	python ../../MusiteDeep_Keras2.0/MusiteDeep/predict.py -input ../fasta_files/all_test_MetOx_$i.fasta -predict-type custom -model-prefix transfer-leayer/models/rephold-STY-nclass1-tl0-lr0001/metox_model_custom_general_STY_$i -output transfer-leayer/results/rephold-STY-nclass1-tl0-lr0001/metox_result_custom_general_STY_$i
done


In [ ]:
%%time
!transfer-leayer/train_bootstrap_custom_model_STY_1_tl_0_lr_0001.sh

#### Fasta files with no initial M

Using nclass=1:

In [6]:
!cat musite-fasta-M/train_bootstrap_custom_model_STY_1.sh

#!/bin/bash

for i in {1..10}
do
	# Train
	# Using repeated holdout strategy
	# Using the custom general model pre-trained on S, T, Y residues
	# Using nclass=1
	# - initializing only the weights in last layer (for transfer learning)
	python ../../MusiteDeep_Keras2.0/MusiteDeep/train_kinase.py -input fasta_files_M/all_train_MetOx_$i.fasta -background-prefix models/pre-train/custom_general_STY -output-prefix musite-fasta-M/models/fine-tune/rephold-STY-nclass1/metox_model_custom_general_STY_$i -residue-types M -nclass 1 -transferlayer 1

	# Test
	python ../../MusiteDeep_Keras2.0/MusiteDeep/predict.py -input fasta_files_M/all_test_MetOx_$i.fasta -predict-type custom -model-prefix musite-fasta-M/models/fine-tune/rephold-STY-nclass1/metox_model_custom_general_STY_$i -output musite-fasta-M/results/rephold-STY-nclass1/metox_result_custom_general_STY_$i
done


In [ ]:
%%time
!musite-fasta-M/train_bootstrap_custom_model_STY_1.sh

## Using bootstrap strategy

Models are fine-tuned on MetOx data.

### Residues S, T

Using nclass=5:

In [1]:
!cat train_bootstrap_BS_custom_model_ST_5.sh

#!/bin/bash

# Store current directory
d=$PWD

for i in {1..100}
do
	# Train
	# Using bootstrap strategy
	# Using the custom general model pre-trained on S, T residues
	# Using nclass=5 (default)
	# - initializing only the weights in last layer (for transfer learning)
	python ../../MusiteDeep_Keras2.0/MusiteDeep/train_kinase.py -input ../fasta_files_bs/all_train_MetOx_bs_$i.fasta -background-prefix models/pre-train/custom_general_ST -output-prefix models/fine-tune/bs-ST-nclass5/metox_model_custom_general_ST_bs_$i -residue-types M -transferlayer 1

	# Test
	python ../../MusiteDeep_Keras2.0/MusiteDeep/predict.py -input ../fasta_files_bs/all_test_MetOx_bs_$i.fasta -predict-type custom -model-prefix models/fine-tune/bs-ST-nclass5/metox_model_custom_general_ST_bs_$i -output results/bs-ST-nclass5/metox_result_custom_general_ST_bs_$i

	# Remove model files
        cd models/fine-tune/bs-ST-nclass5/
        ls | grep metox_model_custom_general_ST_bs_.*_HDF5 | xargs -d"\n" rm
        cd $d
done

In [ ]:
%%time
!./train_bootstrap_BS_custom_model_ST_5.sh

Using nclass=1:

In [4]:
!cat train_bootstrap_BS_custom_model_ST_1.sh

#!/bin/bash

for i in {1..100}
do
	# Train
	# Using bootstrap strategy
	# Using the custom general model pre-trained on S, T residues
	# Using nclass=1
	# - initializing only the weights in last layer (for transfer learning)
	python ../../MusiteDeep_Keras2.0/MusiteDeep/train_kinase.py -input ../fasta_files_bs/all_train_MetOx_bs_$i.fasta -background-prefix models/pre-train/custom_general_ST -output-prefix models/fine-tune/bs-ST-nclass1/metox_model_custom_general_ST_bs_$i -residue-types M -nclass 1 -transferlayer 1

	# Test
	python ../../MusiteDeep_Keras2.0/MusiteDeep/predict.py -input ../fasta_files_bs/all_test_MetOx_bs_$i.fasta -predict-type custom -model-prefix models/fine-tune/bs-ST-nclass1/metox_model_custom_general_ST_bs_$i -output results/bs-ST-nclass1/metox_result_custom_general_ST_bs_$i
done

In [ ]:
%%time
!./train_bootstrap_BS_custom_model_ST_1.sh

#### Fasta files with no initial M

Using nclass=1:

In [1]:
!cat musite-fasta-M/train_bootstrap_BS_custom_model_ST_1.sh

#!/bin/bash

# Store current directory
d=$PWD

for i in {1..100}
do
	# Train
	# Using bootstrap strategy
	# Using the custom general model pre-trained on S, T residues
	# Using nclass=1
	# - initializing only the weights in last layer (for transfer learning)
	python ../../MusiteDeep_Keras2.0/MusiteDeep/train_kinase.py -input fasta_files_bs_M/all_train_MetOx_bs_$i.fasta -background-prefix models/pre-train/custom_general_ST -output-prefix musite-fasta-M/models/fine-tune/bs-ST-nclass1/metox_model_custom_general_ST_bs_$i -residue-types M -nclass 1 -transferlayer 1

	# Test
	python ../../MusiteDeep_Keras2.0/MusiteDeep/predict.py -input fasta_files_bs_M/all_test_MetOx_bs_$i.fasta -predict-type custom -model-prefix musite-fasta-M/models/fine-tune/bs-ST-nclass1/metox_model_custom_general_ST_bs_$i -output musite-fasta-M/results/bs-ST-nclass1/metox_result_custom_general_ST_bs_$i

	# Remove model files
        cd musite-fasta-M/models/fine-tune/bs-ST-nclass1/
        ls | grep metox_model_custom_

In [ ]:
%%time
!musite-fasta-M/train_bootstrap_BS_custom_model_ST_1.sh

### Residues S, T, Y

Using nclass=5:

In [5]:
!cat train_bootstrap_BS_custom_model_STY_5.sh

#!/bin/bash

# Store current directory
d=$PWD

for i in {1..100}
do
	# Train
	# Using bootstrap strategy
	# Using the custom general model pre-trained on S, T, Y residues
	# Using nclass=5 (default)
	# - initializing only the weights in last layer (for transfer learning)
	python ../../MusiteDeep_Keras2.0/MusiteDeep/train_kinase.py -input ../fasta_files_bs/all_train_MetOx_bs_$i.fasta -background-prefix models/pre-train/custom_general_STY -output-prefix models/fine-tune/bs-STY-nclass5/metox_model_custom_general_STY_bs_$i -residue-types M -transferlayer 1

	# Test
	python ../../MusiteDeep_Keras2.0/MusiteDeep/predict.py -input ../fasta_files_bs/all_test_MetOx_bs_$i.fasta -predict-type custom -model-prefix models/fine-tune/bs-STY-nclass5/metox_model_custom_general_STY_bs_$i -output results/bs-STY-nclass5/metox_result_custom_general_STY_bs_$i

	# Remove model files
	cd models/fine-tune/bs-STY-nclass5/
	ls | grep metox_model_custom_general_STY_bs_.*_HDF5 | xargs -d"\n" rm
	cd $d
done

# Remov

In [ ]:
%%time
!./train_bootstrap_BS_custom_model_STY_5.sh

Using nclass=1:

In [1]:
!cat train_bootstrap_BS_custom_model_STY_1.sh

#!/bin/bash

# Store current directory
d=$PWD

for i in {1..100}
do
	# Train
	# Using bootstrap strategy
	# Using the custom general model pre-trained on S, T, Y residues
	# Using nclass=1
	# - initializing only the weights in last layer (for transfer learning)
	python ../../MusiteDeep_Keras2.0/MusiteDeep/train_kinase.py -input ../fasta_files_bs/all_train_MetOx_bs_$i.fasta -background-prefix models/pre-train/custom_general_STY -output-prefix models/fine-tune/bs-STY-nclass1/metox_model_custom_general_STY_bs_$i -residue-types M -nclass 1 -transferlayer 1

	# Test
	python ../../MusiteDeep_Keras2.0/MusiteDeep/predict.py -input ../fasta_files_bs/all_test_MetOx_bs_$i.fasta -predict-type custom -model-prefix models/fine-tune/bs-STY-nclass1/metox_model_custom_general_STY_bs_$i -output results/bs-STY-nclass1/metox_result_custom_general_STY_bs_$i

	# Remove model files
	cd models/fine-tune/bs-STY-nclass1/
	ls | grep metox_model_custom_general_STY_bs_.*_HDF5 | xargs -d"\n" rm
	cd $d
done

# Remov

In [ ]:
%%time
!./train_bootstrap_BS_custom_model_STY_1.sh

#### Transfer-leayer and learning-rate

Using lr=0.00075

In [40]:
!cat lr/train_bootstrap_BS_custom_model_STY_1_lr_00075.sh

#!/bin/bash

# Store current directory
d=$PWD

for i in {1..100}
do
	# Train
	# Using bootstrap strategy
	# Using the custom general model pre-trained on S, T, Y residues
	# Using nclass=1 and lr=0.00075
	# - initializing only the weights in last layer (for transfer learning)
	python ../../MusiteDeep_Keras2.0/MusiteDeep/train_kinase.py -input ../fasta_files_bs/all_train_MetOx_bs_$i.fasta -background-prefix models/pre-train/custom_general_STY -output-prefix lr/models/fine-tune/bs-STY-nclass1-lr00075/metox_model_custom_general_STY_bs_$i -residue-types M -nclass 1 -transferlayer 1 -lr 0.00075

	# Test
	python ../../MusiteDeep_Keras2.0/MusiteDeep/predict.py -input ../fasta_files_bs/all_test_MetOx_bs_$i.fasta -predict-type custom -model-prefix lr/models/fine-tune/bs-STY-nclass1-lr00075/metox_model_custom_general_STY_bs_$i -output lr/results/bs-STY-nclass1-lr00075/metox_result_custom_general_STY_bs_$i

	# Remove model files
	cd lr/models/fine-tune/bs-STY-nclass1-lr00075/
	ls | grep metox_mod

In [ ]:
%%time
!lr/train_bootstrap_BS_custom_model_STY_1_lr_00075.sh

Using lr=0.0005

In [2]:
!cat lr/train_bootstrap_BS_custom_model_STY_1_lr_0005.sh

#!/bin/bash

# Store current directory
d=$PWD

for i in {1..100}
do
	# Train
	# Using bootstrap strategy
	# Using the custom general model pre-trained on S, T, Y residues
	# Using nclass=1 and lr=0.0005
	# - initializing only the weights in last layer (for transfer learning)
	python ../../MusiteDeep_Keras2.0/MusiteDeep/train_kinase.py -input ../fasta_files_bs/all_train_MetOx_bs_$i.fasta -background-prefix models/pre-train/custom_general_STY -output-prefix lr/models/fine-tune/bs-STY-nclass1-lr0005/metox_model_custom_general_STY_bs_$i -residue-types M -nclass 1 -transferlayer 1 -lr 0.0005

	# Test
	python ../../MusiteDeep_Keras2.0/MusiteDeep/predict.py -input ../fasta_files_bs/all_test_MetOx_bs_$i.fasta -predict-type custom -model-prefix lr/models/fine-tune/bs-STY-nclass1-lr0005/metox_model_custom_general_STY_bs_$i -output lr/results/bs-STY-nclass1-lr0005/metox_result_custom_general_STY_bs_$i

	# Remove model files
	cd lr/models/fine-tune/bs-STY-nclass1-lr0005/
	ls | grep metox_model_cus

In [ ]:
%%time
!lr/train_bootstrap_BS_custom_model_STY_1_lr_0005.sh

Using lr=0.00025:

In [5]:
!cat lr/train_bootstrap_BS_custom_model_STY_1_lr_00025.sh

#!/bin/bash

# Store current directory
d=$PWD

for i in {1..100}
do
	# Train
	# Using bootstrap strategy
	# Using the custom general model pre-trained on S, T, Y residues
	# Using nclass=1 and lr=0.00025
	# - initializing only the weights in last layer (for transfer learning)
	python ../../MusiteDeep_Keras2.0/MusiteDeep/train_kinase.py -input ../fasta_files_bs/all_train_MetOx_bs_$i.fasta -background-prefix models/pre-train/custom_general_STY -output-prefix lr/models/fine-tune/bs-STY-nclass1-lr00025/metox_model_custom_general_STY_bs_$i -residue-types M -nclass 1 -transferlayer 1 -lr 0.00025

	# Test
	python ../../MusiteDeep_Keras2.0/MusiteDeep/predict.py -input ../fasta_files_bs/all_test_MetOx_bs_$i.fasta -predict-type custom -model-prefix lr/models/fine-tune/bs-STY-nclass1-lr00025/metox_model_custom_general_STY_bs_$i -output lr/results/bs-STY-nclass1-lr00025/metox_result_custom_general_STY_bs_$i

	# Remove model files
	cd lr/models/fine-tune/bs-STY-nclass1-lr00025/
	ls | grep metox_mod

In [ ]:
%%time
!lr/train_bootstrap_BS_custom_model_STY_1_lr_00025.sh

Using lr=0.0001:

In [7]:
!cat lr/train_bootstrap_BS_custom_model_STY_1_lr_0001.sh

#!/bin/bash

# Store current directory
d=$PWD

for i in {1..100}
do
	# Train
	# Using bootstrap strategy
	# Using the custom general model pre-trained on S, T, Y residues
	# Using nclass=1 and lr=0.0001
	# - initializing only the weights in last layer (for transfer learning)
	python ../../MusiteDeep_Keras2.0/MusiteDeep/train_kinase.py -input ../fasta_files_bs/all_train_MetOx_bs_$i.fasta -background-prefix models/pre-train/custom_general_STY -output-prefix lr/models/fine-tune/bs-STY-nclass1-lr0001/metox_model_custom_general_STY_bs_$i -residue-types M -nclass 1 -transferlayer 1 -lr 0.0001

	# Test
	python ../../MusiteDeep_Keras2.0/MusiteDeep/predict.py -input ../fasta_files_bs/all_test_MetOx_bs_$i.fasta -predict-type custom -model-prefix lr/models/fine-tune/bs-STY-nclass1-lr0001/metox_model_custom_general_STY_bs_$i -output lr/results/bs-STY-nclass1-lr0001/metox_result_custom_general_STY_bs_$i

	# Remove model files
	cd lr/models/fine-tune/bs-STY-nclass1-lr0001/
	ls | grep metox_model_cus

In [ ]:
%%time
!lr/train_bootstrap_BS_custom_model_STY_1_lr_0001.sh

Using lr=0.00001:

In [2]:
!cat lr/train_bootstrap_BS_custom_model_STY_1_lr_00001.sh

#!/bin/bash

# Store current directory
d=$PWD

for i in {1..100}
do
	# Train
	# Using bootstrap strategy
	# Using the custom general model pre-trained on S, T, Y residues
	# Using nclass=1 and lr=0.00001
	# - initializing only the weights in last layer (for transfer learning)
	python ../../MusiteDeep_Keras2.0/MusiteDeep/train_kinase.py -input ../fasta_files_bs/all_train_MetOx_bs_$i.fasta -background-prefix models/pre-train/custom_general_STY -output-prefix lr/models/fine-tune/bs-STY-nclass1-lr00001/metox_model_custom_general_STY_bs_$i -residue-types M -nclass 1 -transferlayer 1 -lr 0.00001

	# Test
	python ../../MusiteDeep_Keras2.0/MusiteDeep/predict.py -input ../fasta_files_bs/all_test_MetOx_bs_$i.fasta -predict-type custom -model-prefix lr/models/fine-tune/bs-STY-nclass1-lr00001/metox_model_custom_general_STY_bs_$i -output lr/results/bs-STY-nclass1-lr00001/metox_result_custom_general_STY_bs_$i

	# Remove model files
	cd lr/models/fine-tune/bs-STY-nclass1-lr00001/
	ls | grep metox_mod

In [ ]:
%%time
!lr/train_bootstrap_BS_custom_model_STY_1_lr_00001.sh

Using lr=0.00125:

In [1]:
!cat lr/train_bootstrap_BS_custom_model_STY_1_lr_00125.sh

#!/bin/bash

# Store current directory
d=$PWD

for i in {1..100}
do
	# Train
	# Using bootstrap strategy
	# Using the custom general model pre-trained on S, T, Y residues
	# Using nclass=1 and lr=0.00125
	# - initializing only the weights in last layer (for transfer learning)
	python ../../MusiteDeep_Keras2.0/MusiteDeep/train_kinase.py -input ../fasta_files_bs/all_train_MetOx_bs_$i.fasta -background-prefix models/pre-train/custom_general_STY -output-prefix lr/models/fine-tune/bs-STY-nclass1-lr00125/metox_model_custom_general_STY_bs_$i -residue-types M -nclass 1 -transferlayer 1 -lr 0.00125

	# Test
	python ../../MusiteDeep_Keras2.0/MusiteDeep/predict.py -input ../fasta_files_bs/all_test_MetOx_bs_$i.fasta -predict-type custom -model-prefix lr/models/fine-tune/bs-STY-nclass1-lr00125/metox_model_custom_general_STY_bs_$i -output lr/results/bs-STY-nclass1-lr00125/metox_result_custom_general_STY_bs_$i

	# Remove model files
	cd lr/models/fine-tune/bs-STY-nclass1-lr00125/
	ls | grep metox_mod

In [ ]:
%%time
!lr/train_bootstrap_BS_custom_model_STY_1_lr_00125.sh

Using lr=0.0015:

In [3]:
!cat lr/train_bootstrap_BS_custom_model_STY_1_lr_0015.sh

#!/bin/bash

# Store current directory
d=$PWD

for i in {1..100}
do
	# Train
	# Using bootstrap strategy
	# Using the custom general model pre-trained on S, T, Y residues
	# Using nclass=1 and lr=0.0015
	# - initializing only the weights in last layer (for transfer learning)
	python ../../MusiteDeep_Keras2.0/MusiteDeep/train_kinase.py -input ../fasta_files_bs/all_train_MetOx_bs_$i.fasta -background-prefix models/pre-train/custom_general_STY -output-prefix lr/models/fine-tune/bs-STY-nclass1-lr0015/metox_model_custom_general_STY_bs_$i -residue-types M -nclass 1 -transferlayer 1 -lr 0.0015

	# Test
	python ../../MusiteDeep_Keras2.0/MusiteDeep/predict.py -input ../fasta_files_bs/all_test_MetOx_bs_$i.fasta -predict-type custom -model-prefix lr/models/fine-tune/bs-STY-nclass1-lr0015/metox_model_custom_general_STY_bs_$i -output lr/results/bs-STY-nclass1-lr0015/metox_result_custom_general_STY_bs_$i

	# Remove model files
	cd lr/models/fine-tune/bs-STY-nclass1-lr0015/
	ls | grep metox_model_cus

In [ ]:
%%time
!lr/train_bootstrap_BS_custom_model_STY_1_lr_0015.sh

Using tl=0:

In [9]:
!cat transfer-leayer/train_bootstrap_BS_custom_model_STY_1_tl_0.sh

#!/bin/bash

# Store current directory
d=$PWD

for i in {1..100}
do
	# Train
	# Using bootstrap strategy
	# Using the custom general model pre-trained on S, T, Y residues
	# Using nclass=1 and transfer-leayer=0
	# - initializing only the weights in last layer (for transfer learning)
	python ../../MusiteDeep_Keras2.0/MusiteDeep/train_kinase.py -input ../fasta_files_bs/all_train_MetOx_bs_$i.fasta -background-prefix models/pre-train/custom_general_STY -output-prefix transfer-leayer/models/bs-STY-nclass1-tl0/metox_model_custom_general_STY_bs_$i -residue-types M -nclass 1 -transferlayer 0

	# Test
	python ../../MusiteDeep_Keras2.0/MusiteDeep/predict.py -input ../fasta_files_bs/all_test_MetOx_bs_$i.fasta -predict-type custom -model-prefix transfer-leayer/models/bs-STY-nclass1-tl0/metox_model_custom_general_STY_bs_$i -output transfer-leayer/results/bs-STY-nclass1-tl0/metox_result_custom_general_STY_bs_$i

	# Remove model files
        cd transfer-leayer/models/bs-STY-nclass1-tl0/
        ls |

In [ ]:
%%time
!transfer-leayer/train_bootstrap_BS_custom_model_STY_1_tl_0.sh

Using tl=3:

In [5]:
!cat transfer-leayer/train_bootstrap_BS_custom_model_STY_1_tl_3.sh

#!/bin/bash

# Store current directory
d=$PWD

for i in {1..100}
do
	# Train
	# Using bootstrap strategy
	# Using the custom general model pre-trained on S, T, Y residues
	# Using nclass=1 and transfer-leayer=3
	# - initializing only the weights in last layer (for transfer learning)
	python ../../MusiteDeep_Keras2.0/MusiteDeep/train_kinase.py -input ../fasta_files_bs/all_train_MetOx_bs_$i.fasta -background-prefix models/pre-train/custom_general_STY -output-prefix transfer-leayer/models/bs-STY-nclass1-tl3/metox_model_custom_general_STY_bs_$i -residue-types M -nclass 1 -transferlayer 3

	# Test
	python ../../MusiteDeep_Keras2.0/MusiteDeep/predict.py -input ../fasta_files_bs/all_test_MetOx_bs_$i.fasta -predict-type custom -model-prefix transfer-leayer/models/bs-STY-nclass1-tl3/metox_model_custom_general_STY_bs_$i -output transfer-leayer/results/bs-STY-nclass1-tl3/metox_result_custom_general_STY_bs_$i

	# Remove model files
        cd transfer-leayer/models/bs-STY-nclass1-tl3/
        ls |

In [ ]:
%%time
!transfer-leayer/train_bootstrap_BS_custom_model_STY_1_tl_3.sh

Using tl=5:

In [10]:
!cat transfer-leayer/train_bootstrap_BS_custom_model_STY_1_tl_5.sh

#!/bin/bash

# Store current directory
d=$PWD

for i in {1..100}
do
	# Train
	# Using bootstrap strategy
	# Using the custom general model pre-trained on S, T, Y residues
	# Using nclass=1 and transfer-leayer=5
	# - initializing only the weights in last layer (for transfer learning)
	python ../../MusiteDeep_Keras2.0/MusiteDeep/train_kinase.py -input ../fasta_files_bs/all_train_MetOx_bs_$i.fasta -background-prefix models/pre-train/custom_general_STY -output-prefix transfer-leayer/models/bs-STY-nclass1-tl5/metox_model_custom_general_STY_bs_$i -residue-types M -nclass 1 -transferlayer 5

	# Test
	python ../../MusiteDeep_Keras2.0/MusiteDeep/predict.py -input ../fasta_files_bs/all_test_MetOx_bs_$i.fasta -predict-type custom -model-prefix transfer-leayer/models/bs-STY-nclass1-tl5/metox_model_custom_general_STY_bs_$i -output transfer-leayer/results/bs-STY-nclass1-tl5/metox_result_custom_general_STY_bs_$i

	# Remove model files
        cd transfer-leayer/models/bs-STY-nclass1-tl5/
        ls |

In [ ]:
%%time
!transfer-leayer/train_bootstrap_BS_custom_model_STY_1_tl_5.sh

Using tl=0 and lr=0.0005:

In [42]:
!cat transfer-leayer/train_bootstrap_BS_custom_model_STY_1_tl_0_lr_0005.sh

#!/bin/bash

# Store current directory
d=$PWD

for i in {1..100}
do
	# Train
	# Using bootstrap strategy
	# Using the custom general model pre-trained on S, T, Y residues
	# Using nclass=1, transfer-leayer=0 and lr=0.0005
	# - initializing only the weights in last layer (for transfer learning)
	python ../../MusiteDeep_Keras2.0/MusiteDeep/train_kinase.py -input ../fasta_files_bs/all_train_MetOx_bs_$i.fasta -background-prefix models/pre-train/custom_general_STY -output-prefix transfer-leayer/models/bs-STY-nclass1-tl0-lr0005/metox_model_custom_general_STY_bs_$i -residue-types M -nclass 1 -transferlayer 0 -lr 0.0005

	# Test
	python ../../MusiteDeep_Keras2.0/MusiteDeep/predict.py -input ../fasta_files_bs/all_test_MetOx_bs_$i.fasta -predict-type custom -model-prefix transfer-leayer/models/bs-STY-nclass1-tl0-lr0005/metox_model_custom_general_STY_bs_$i -output transfer-leayer/results/bs-STY-nclass1-tl0-lr0005/metox_result_custom_general_STY_bs_$i

	# Remove model files
        cd transfer-lea

In [ ]:
%%time
!transfer-leayer/train_bootstrap_BS_custom_model_STY_1_tl_0_lr_0005.sh

Using tl=0 and lr=0.00025:

In [5]:
!cat transfer-leayer/train_bootstrap_BS_custom_model_STY_1_tl_0_lr_00025.sh

#!/bin/bash

# Store current directory
d=$PWD

for i in {1..100}
do
	# Train
	# Using bootstrap strategy
	# Using the custom general model pre-trained on S, T, Y residues
	# Using nclass=1, transfer-leayer=0 and lr=0.00025
	# - initializing only the weights in last layer (for transfer learning)
	python ../../MusiteDeep_Keras2.0/MusiteDeep/train_kinase.py -input ../fasta_files_bs/all_train_MetOx_bs_$i.fasta -background-prefix models/pre-train/custom_general_STY -output-prefix transfer-leayer/models/bs-STY-nclass1-tl0-lr00025/metox_model_custom_general_STY_bs_$i -residue-types M -nclass 1 -transferlayer 0 -lr 0.00025

	# Test
	python ../../MusiteDeep_Keras2.0/MusiteDeep/predict.py -input ../fasta_files_bs/all_test_MetOx_bs_$i.fasta -predict-type custom -model-prefix transfer-leayer/models/bs-STY-nclass1-tl0-lr00025/metox_model_custom_general_STY_bs_$i -output transfer-leayer/results/bs-STY-nclass1-tl0-lr00025/metox_result_custom_general_STY_bs_$i

	# Remove model files
        cd transfe

In [ ]:
%%time
!transfer-leayer/train_bootstrap_BS_custom_model_STY_1_tl_0_lr_00025.sh

Using tl=0 and lr=0.0001:

In [5]:
!cat transfer-leayer/train_bootstrap_BS_custom_model_STY_1_tl_0_lr_0001.sh

#!/bin/bash

# Store current directory
d=$PWD

for i in {1..100}
do
	# Train
	# Using bootstrap strategy
	# Using the custom general model pre-trained on S, T, Y residues
	# Using nclass=1, transfer-leayer=0 and lr=0.0001
	# - initializing only the weights in last layer (for transfer learning)
	python ../../MusiteDeep_Keras2.0/MusiteDeep/train_kinase.py -input ../fasta_files_bs/all_train_MetOx_bs_$i.fasta -background-prefix models/pre-train/custom_general_STY -output-prefix transfer-leayer/models/bs-STY-nclass1-tl0-lr0001/metox_model_custom_general_STY_bs_$i -residue-types M -nclass 1 -transferlayer 0 -lr 0.0001

	# Test
	python ../../MusiteDeep_Keras2.0/MusiteDeep/predict.py -input ../fasta_files_bs/all_test_MetOx_bs_$i.fasta -predict-type custom -model-prefix transfer-leayer/models/bs-STY-nclass1-tl0-lr0001/metox_model_custom_general_STY_bs_$i -output transfer-leayer/results/bs-STY-nclass1-tl0-lr0001/metox_result_custom_general_STY_bs_$i

	# Remove model files
        cd transfer-lea

In [ ]:
%%time
!transfer-leayer/train_bootstrap_BS_custom_model_STY_1_tl_0_lr_0001.sh

Using tl=3 and lr=0.0005:

In [1]:
!cat transfer-leayer/train_bootstrap_BS_custom_model_STY_1_tl_3_lr_0005.sh

#!/bin/bash

# Store current directory
d=$PWD

for i in {1..100}
do
	# Train
	# Using bootstrap strategy
	# Using the custom general model pre-trained on S, T, Y residues
	# Using nclass=1, transfer-leayer=3 and lr=0.0005
	# - initializing only the weights in last layer (for transfer learning)
	python ../../MusiteDeep_Keras2.0/MusiteDeep/train_kinase.py -input ../fasta_files_bs/all_train_MetOx_bs_$i.fasta -background-prefix models/pre-train/custom_general_STY -output-prefix transfer-leayer/models/bs-STY-nclass1-tl3-lr0005/metox_model_custom_general_STY_bs_$i -residue-types M -nclass 1 -transferlayer 3 -lr 0.0005

	# Test
	python ../../MusiteDeep_Keras2.0/MusiteDeep/predict.py -input ../fasta_files_bs/all_test_MetOx_bs_$i.fasta -predict-type custom -model-prefix transfer-leayer/models/bs-STY-nclass1-tl3-lr0005/metox_model_custom_general_STY_bs_$i -output transfer-leayer/results/bs-STY-nclass1-tl3-lr0005/metox_result_custom_general_STY_bs_$i

	# Remove model files
        cd transfer-lea

In [ ]:
%%time
!transfer-leayer/train_bootstrap_BS_custom_model_STY_1_tl_3_lr_0005.sh

Using tl=3 and lr=0.00025:

In [3]:
!cat transfer-leayer/train_bootstrap_BS_custom_model_STY_1_tl_3_lr_00025.sh

#!/bin/bash

# Store current directory
d=$PWD

for i in {1..100}
do
	# Train
	# Using bootstrap strategy
	# Using the custom general model pre-trained on S, T, Y residues
	# Using nclass=1, transfer-leayer=3 and lr=0.00025
	# - initializing only the weights in last layer (for transfer learning)
	python ../../MusiteDeep_Keras2.0/MusiteDeep/train_kinase.py -input ../fasta_files_bs/all_train_MetOx_bs_$i.fasta -background-prefix models/pre-train/custom_general_STY -output-prefix transfer-leayer/models/bs-STY-nclass1-tl3-lr00025/metox_model_custom_general_STY_bs_$i -residue-types M -nclass 1 -transferlayer 3 -lr 0.00025

	# Test
	python ../../MusiteDeep_Keras2.0/MusiteDeep/predict.py -input ../fasta_files_bs/all_test_MetOx_bs_$i.fasta -predict-type custom -model-prefix transfer-leayer/models/bs-STY-nclass1-tl3-lr00025/metox_model_custom_general_STY_bs_$i -output transfer-leayer/results/bs-STY-nclass1-tl3-lr00025/metox_result_custom_general_STY_bs_$i

	# Remove model files
        cd transfe

In [ ]:
%%time
!transfer-leayer/train_bootstrap_BS_custom_model_STY_1_tl_3_lr_00025.sh

#### Using all phosphorylation data

Using nclass=1 and STY-residues:

In [1]:
!cat all-phos-data/train_bootstrap_BS_custom_model_STY_1.sh

#!/bin/bash

# Store current directory
d=$PWD

for i in {1..100}
do
	# Train
	# Using bootstrap strategy
	# Using the custom general model pre-trained on both training and test S, T, Y residues
	# Using nclass=1
	# - initializing only the weights in last layer (for transfer learning)
	python ../../MusiteDeep_Keras2.0/MusiteDeep/train_kinase.py -input ../fasta_files_bs/all_train_MetOx_bs_$i.fasta -background-prefix all-phos-data/models/pre-train/custom_general_STY -output-prefix all-phos-data/models/fine-tune/bs-STY-nclass1/metox_model_custom_general_STY_bs_$i -residue-types M -nclass 1 -transferlayer 1

	# Test
	python ../../MusiteDeep_Keras2.0/MusiteDeep/predict.py -input ../fasta_files_bs/all_test_MetOx_bs_$i.fasta -predict-type custom -model-prefix all-phos-data/models/fine-tune/bs-STY-nclass1/metox_model_custom_general_STY_bs_$i -output all-phos-data/results/bs-STY-nclass1/metox_result_custom_general_STY_bs_$i

	# Remove model files
	cd all-phos-data/models/fine-tune/bs-STY-nclass1

In [ ]:
%%time
!all-phos-data/train_bootstrap_BS_custom_model_STY_1.sh

Using nclass=1 and ST-residues:

In [11]:
!cat all-phos-data/train_bootstrap_BS_custom_model.sh

#!/bin/bash


# ----- Arguments management -----

while [[ $# -gt 0 ]]
do

key="$1"
case $key in
    -l|--learning-rate)
    LR=$2
    shift # past argument
    shift # past value
    ;;
    -t|--transfer-leayer)
    TL=$2
    shift # past argument
    shift # past value
    ;;
    -n|--nclass)
    N=$2
    shift # past argument
    shift # past value
    ;;
    -b|--background)
    B=$2
    shift # past argument
    shift # past value
    ;;
    -p|--phospho-residue)
    PH=$2
    shift # past argument
    shift # past value
    ;;
    -f|--fine-output)
    FT=$2
    shift # past argument
    shift # past value
    ;;
    -r|--result-output)
    RES=$2
    shift # past argument
    shift # past value
    ;;
    -h|--help)
	echo "Usage: $0 [OPTION]...
Example: $0 -l 0.003 -t 5

Arguments:
 -l, --learning-rate=NUM	Learning rate value used when fine-tuning
 				the model. It should be a FLOAT value.
 				Default value is 0.001
 -t, --transfer-leayer=NUM	Transfer-leayer value used when fi

In [ ]:
%%time
!all-phos-data/train_bootstrap_BS_custom_model.sh -p ST

##### Transfer-leayer and learning-rate

Using lr=0.00075

In [1]:
!cat all-phos-data/train_bootstrap_BS_custom_model_STY.sh

#!/bin/bash


# ----- Arguments management -----

while [[ $# -gt 0 ]]
do

key="$1"
case $key in
    -l|--learning-rate)
    LR=$2
    shift # past argument
    shift # past value
    ;;
    -t|--transfer-leayer)
    TL=$2
    shift # past argument
    shift # past value
    ;;
    -n|--nclass)
    N=$2
    shift # past argument
    shift # past value
    ;;
    -h|--help)
	echo "Usage: $0 [OPTION]...
Example: $0 -l 0.003 -t 5

Arguments:
 -l, --learning-rate=NUM	Learning rate value used when fine-tuning
 				the model. It should be a FLOAT value.
 				Default value is 0.001
 -t, --transfer-leayer=NUM	Transfer-leayer value used when fine-tuning 
 				the model. It should be an INTEGER value.
 				Default value is 1
 -n, --nclass=NUM		Number of class value used when fine-tuning 
 				the model. It should be an INTEGER value.
 				Default value is 1
 -h, --help			It shows this help and exits"
	exit
	;;
    *)    # unknown option
	echo "ERROR: $1 is not a valid option. Use -h (--help) to 

In [ ]:
%%time
!all-phos-data/train_bootstrap_BS_custom_model_STY.sh -l 0.00075

Using lr=0.0005

In [ ]:
%%time
!all-phos-data/train_bootstrap_BS_custom_model_STY.sh -l 0.0005

Using lr=0.00025

In [ ]:
%%time
!all-phos-data/train_bootstrap_BS_custom_model_STY.sh -l 0.00025

Using lr=0.00125

In [ ]:
%%time
!all-phos-data/train_bootstrap_BS_custom_model_STY.sh -l 0.00125

Using lr=0.0015

In [ ]:
%%time
!all-phos-data/train_bootstrap_BS_custom_model_STY.sh -l 0.0015

Using tl=0:

In [ ]:
%%time
!all-phos-data/train_bootstrap_BS_custom_model_STY.sh -t 0

Using tl=3:

In [ ]:
%%time
!all-phos-data/train_bootstrap_BS_custom_model.sh -t 3

Using tl=5:

In [ ]:
%%time
!all-phos-data/train_bootstrap_BS_custom_model_STY.sh -t 5

Using lr=0.00025 and tl=3:

In [ ]:
%%time
!all-phos-data/train_bootstrap_BS_custom_model.sh -l 0.00025 -t 3

Using lr=0.00025 and tl=0:

In [ ]:
%%time
!all-phos-data/train_bootstrap_BS_custom_model.sh -l 0.00025 -t 0

Using lr=0.0005 and tl=3:

In [ ]:
%%time
!all-phos-data/train_bootstrap_BS_custom_model.sh -l 0.0005 -t 3

Using lr=0.0005 and tl=0:

In [ ]:
%%time
!all-phos-data/train_bootstrap_BS_custom_model.sh -l 0.0005 -t 0

##### Fasta files with no initial M

Using nclass=1:

In [3]:
!cat musite-fasta-M/train_bootstrap_BS_custom_model_STY_1.sh

#!/bin/bash

# Store current directory
d=$PWD

for i in {1..100}
do
	# Train
	# Using bootstrap strategy
	# Using the custom general model pre-trained on S, T, Y residues
	# Using nclass=1
	# - initializing only the weights in last layer (for transfer learning)
	python ../../MusiteDeep_Keras2.0/MusiteDeep/train_kinase.py -input fasta_files_bs_M/all_train_MetOx_bs_$i.fasta -background-prefix models/pre-train/custom_general_STY -output-prefix musite-fasta-M/models/fine-tune/bs-STY-nclass1/metox_model_custom_general_STY_bs_$i -residue-types M -nclass 1 -transferlayer 1

	# Test
	python ../../MusiteDeep_Keras2.0/MusiteDeep/predict.py -input fasta_files_bs_M/all_test_MetOx_bs_$i.fasta -predict-type custom -model-prefix musite-fasta-M/models/fine-tune/bs-STY-nclass1/metox_model_custom_general_STY_bs_$i -output musite-fasta-M/results/bs-STY-nclass1/metox_result_custom_general_STY_bs_$i

	# Remove model files
        cd musite-fasta-M/models/fine-tune/bs-STY-nclass1/
        ls | grep metox_mo

In [ ]:
%%time
!musite-fasta-M/train_bootstrap_BS_custom_model_STY_1.sh

##### Transfer-leayer and learning-rate

Using lr=0.00075

In [4]:
!cat musite-fasta-M/train_bootstrap_BS_custom_model_STY_1_lr_00075.sh

#!/bin/bash

# Store current directory
d=$PWD

for i in {1..100}
do
	# Train
	# Using bootstrap strategy
	# Using the custom general model pre-trained on S, T, Y residues
	# Using nclass=1 and lr=0.00075
	# - initializing only the weights in last layer (for transfer learning)
	python ../../MusiteDeep_Keras2.0/MusiteDeep/train_kinase.py -input fasta_files_bs_M/all_train_MetOx_bs_$i.fasta -background-prefix models/pre-train/custom_general_STY -output-prefix musite-fasta-M/models/fine-tune/bs-STY-nclass1-lr00075/metox_model_custom_general_STY_bs_$i -residue-types M -nclass 1 -transferlayer 1 -lr 0.00075

	# Test
	python ../../MusiteDeep_Keras2.0/MusiteDeep/predict.py -input fasta_files_bs_M/all_test_MetOx_bs_$i.fasta -predict-type custom -model-prefix musite-fasta-M/models/fine-tune/bs-STY-nclass1-lr00075/metox_model_custom_general_STY_bs_$i -output musite-fasta-M/results/bs-STY-nclass1-lr00075/metox_result_custom_general_STY_bs_$i

	# Remove model files
	cd musite-fasta-M/models/fine-tun

In [ ]:
%%time
!musite-fasta-M/train_bootstrap_BS_custom_model_STY_1_lr_00075.sh

# MetOx prediction using DeepBind

In this notebook, we will use the DeepBind model to try to solve the methionine-oxidation problem. We will pre-train DeepBind on general phosphorylation data, and then we will fine-tune it using our methionine-oxidation dataset.

## Custom general model

We now pre-train a custom general model on the general phosphorylation dataset.

General model pre-trained on S, T residues:

In [ ]:
%%time
!python ../../MusiteDeep_Keras2.0/MusiteDeep/train_general.py -input ../../testdata/training_proteins_nonredundant_STY.fasta -cnn 'DeepBind' -output-prefix deepbind/models/pre-train/deep_bind_ST -residue-types S,T -nclass=5

General model pre-trained on S, T, Y residues:

In [ ]:
%%time
!python ../../MusiteDeep_Keras2.0/MusiteDeep/train_general.py -input ../../testdata/training_proteins_nonredundant_STY.fasta -output-prefix models/pre-train/custom_general_STY -residue-types S,T,Y -nclass=5

### Using repeated holdout strategy

Models are fine-tuned on MetOx data.

#### Residues S, T

Using nclass=1, 

In [4]:
!cat deepbind/train_bootstrap_deep_bind_ST_1.sh

#!/bin/bash

for i in {1..10}
do
	# Train
	# Using repeated holdout strategy
	# Using the DeepBind pre-trained on S, T residues
	# Using nclass=1
	# - initializing only the weights in last layer (for transfer learning)
	python ../../MusiteDeep_Keras2.0/MusiteDeep/train_kinase.py -input ../fasta_files/all_train_MetOx_$i.fasta -cnn DeepBind -background-prefix deepbind/models/pre-train/deep_bind_ST -output-prefix deepbind/models/fine-tune/rephold-ST-nclass1/metox_model_deep_bind_ST_$i -residue-types M -nclass 1 -transferlayer 1

	# Test
	python ../../MusiteDeep_Keras2.0/MusiteDeep/predict.py -input ../fasta_files/all_test_MetOx_$i.fasta -cnn DeepBind -predict-type custom -model-prefix deepbind/models/fine-tune/rephold-ST-nclass1/metox_model_deep_bind_ST_$i -output deepbind/results/rephold-ST-nclass1/metox_result_deep_bind_ST_$i
done


In [ ]:
%%time
!deepbind/train_bootstrap_deep_bind_ST_1.sh